In [3]:
#-*- coding:utf-8 -*-
from commonTool import *


In [2]:
# SP-1. 워킹 데이만 뽑아 저장하는 스크립트 실행
workingDayPath = outputRawPath + 'workingDay' + os.path.sep
mkdir(workingDayPath)

currentYear = today()[0:4]

scriptPathName = crawlegoScriptPath + 'BASIS_DATE.xml'
for year in range(2012, int(currentYear) + 1):
    if year != int(currentYear) and os.path.exists(workingDayPath + 'VD-' + str(year) + '.txt'):
        continue

    parameter = {
        'YEAR': str(year),
        'OUT_PATH': workingDayPath,
        'DO_SERVER': '13.124.29.70'
    }

    print(year, 'ret code', runDashScript(scriptPathName, parameter))

java -Dfile.encoding=utf8 -Duser.timezone=GMT -jar D:\work\jupyter\krx\jar\crawlego-1.0.0.jar D:\work\jupyter\krx\script\ZTMP_DATE.xml YEAR=2021 OUT_PATH=D:\work\jupyter\krx\output\workingDay\ DO_SERVER=13.124.29.70
2021 ret code 0


In [3]:
# SP-2. 워킹 데이 중 달별로 기준일 추출하는 스크립트 실행
# 매달 첫 째날, 둘 째날, 마지막 날, 마지막 전날 총 4일을 저장함
# resourceDir에 BASIS_DAYs.txt와 주가 쿼리를 위한 날짜 조건을 담고 있는 BASIS-CONDITION.txt 파일 생성

scriptPathName = crawlegoScriptPath + 'BASIS_DATE_MINMAX.xml'
parameter = {
    'IN_PATH': workingDayPath,
    'OUT_PATH': resourceDir
}

print('ret code', runDashScript(scriptPathName, parameter))

java -Dfile.encoding=utf8 -Duser.timezone=GMT -jar D:\work\jupyter\krx\jar\crawlego-1.0.0.jar D:\work\jupyter\krx\script\ZTMP_DATE_MINMAX.xml IN_PATH=D:\work\jupyter\krx\output\workingDay\ OUT_PATH=D:\work\jupyter\krx\resource\
ret code 0


In [6]:
# SP-3. SP-2의 BASIS-CONDITION.txt 파일을 읽어 해당 날짜의 주가를 쿼리하여 임시폴더에 저장함.

scriptPathName = crawlegoScriptPath + 'BASIS_STOCK_PRICE.xml'

bf = open(resourceDir + 'BASIS-CONDITION.txt', 'r', encoding='utf-8')

lineText = bf.readline() # 제목

while True:
    lineText = bf.readline()
    if not lineText:
        break
        
    lineText = lineText.strip()
    if len(lineText) <= 0:
        continue
        
    dayData = lineText.split('\t')
    
    # 금년만 계산하려면 아래 라인 살리기. 이전 년도는 BASIS가 바뀌지 않는한 변함 없음
    # if dayData[0] != currentYear: continue
    
    parameter = {
        'YEAR': dayData[0],
        'COND': dayData[1],
        'OUT_PATH': temporaryPath,
        'DO_SERVER': '13.124.29.70'
    }

    print(dayData[0], 'ret code', runDashScript(scriptPathName, parameter))

bf.close()

java -Dfile.encoding=utf8 -Duser.timezone=GMT -jar D:\work\jupyter\krx\jar\crawlego-1.0.0.jar D:\work\jupyter\krx\script\ZTMP_STOCK_PRICE.xml YEAR=2021 OUT_PATH=D:\work\jupyter\krx\intermediate\ COND='20210104','20210105','20210128','20210129','20210201','20210202','20210225','20210226','20210302','20210303','20210330','20210331','20210401','20210402','20210429','20210430','20210503','20210504','20210528','20210531','20210601','20210602','20210617','20210618' DO_SERVER=13.124.29.70
2021 ret code 0


In [8]:
# SP-4. 기준이 되는 날짜의 주가를 하나로 합쳐 DB에 업로딩.

scriptPathName = crawlegoScriptPath + 'BASIS_MERGE_PRICE.xml'

parameter = {
    'IN_PATH': temporaryPath,
    'DO_SERVER': '13.124.29.70'
}

print('ret code', runDashScript(scriptPathName, parameter))

java -Dfile.encoding=utf8 -Duser.timezone=GMT -jar D:\work\jupyter\krx\jar\crawlego-1.0.0.jar D:\work\jupyter\krx\script\ZTMP_MERGE_PRICE.xml IN_PATH=D:\work\jupyter\krx\intermediate\ DO_SERVER=13.124.29.70
ret code 1
